# Tutorial de Ciencia de Datos (CC408) 2024

## Tutorial 10 - Bonus: Funciones

**Objetivo**: revisar algunas técnicas de programación para simplificar nuestro código en el futuro y aprender a iterar sobre modelos

#### Importamos los módulos necesarios

In [1]:
import os  
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt  
import statsmodels.api as sm     
import seaborn as sns

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

In [2]:
eph_file = "usu_individual_T124.xlsx"
ad_equiv_xls = "tabla_adulto_equiv_long.xlsx"

In [ ]:
# Abrir la base
eph = pd.read_excel(eph_file)

# Examinamos la base
print(eph.info()) 

# Dimensiones (nro. de filas y de columnas)
num_rows, num_columns = eph.shape

print(f"\nNúmero de filas: {num_rows}") #46050 obs
print(f"Número de columnas: {num_columns}")

print("\n", eph.head()) 

In [ ]:
eph.head()

In [5]:
def preprocesar_eph(eph_file, ad_equiv_xls):    
   
    '''
    Preprocesamiento de la EPH:
        - filtrar solo valores de GBA y CABA
        - filtrar solo valores con sentido
        - armar bases respondieron y no respondieron
    params:
      eph_file: ruta al archivo de la EPH
      ad_equiv_xls: ruta al archivo con la tabla de adultos equivalentes
    
    return:
      respondieron, norespondieron
    ''' 

    eph = pd.read_excel(eph_file)
    
    # Solo GBA y CABA
    eph_ba = eph.loc[eph['AGLOMERADO'].isin([32,33])]
    # Describe
    eph_ba[["CH04", "CH07", "CH08", "NIVEL_ED", "ESTADO", "CAT_INAC", "IPCF", "ITF", "CH06"]].describe()
    
    # Filtrar valores sin sentido
    eph_ba_filtrada=eph_ba[(eph_ba["CH06"]>0)]
    eph_ba_filtrada[["CH04", "CH07", "CH08", "NIVEL_ED", "ESTADO", "CAT_INAC", "IPCF", "ITF", "CH06"]].describe()
    
    tabla = pd.read_excel(ad_equiv_xls)
    eph_ba_filtrada = eph_ba_filtrada.merge(tabla, on=["CH04", "CH06"], how="left")
    
    ad_equiv_hogar = eph_ba_filtrada[["ad_equiv", "CODUSU", "NRO_HOGAR"]].groupby(by=["CODUSU", "NRO_HOGAR"]).agg({"ad_equiv":"sum"})
    ad_equiv_hogar.reset_index(inplace = True)
    ad_equiv_hogar.rename({"ad_equiv" : "ad_equiv_hogar"}, inplace = True, axis="columns")
    
    eph_ba_filtrada = eph_ba_filtrada.merge(ad_equiv_hogar, on=["CODUSU", "NRO_HOGAR"], how="left")
    
    respondieron = eph_ba_filtrada[eph_ba_filtrada["ITF"] != 0]
    respondieron["ingreso_necesario"] = 132853.3 * respondieron["ad_equiv_hogar"]
    respondieron["pobre"] = np.where(respondieron["ITF"] < respondieron["ingreso_necesario"], 1, 0)
    
    no_respondieron = eph_ba_filtrada[eph_ba_filtrada["ITF"] == 0]
    
    return respondieron, no_respondieron

In [6]:
def generar_base_prediccion(df):

    '''
      params:
        df: DataFrame de Pandas con la EPH preprocesada. 
      return:
        Si df contiene una columna llamada "pobre" devuelve dos datasets: X (los predictores) e y (las etiquetas). 
        Si no, solamente devuelve X.
    '''

    columns_to_drop = ['PP06C', 'PP06D', 'PP08D1', 'PP08D4', 'PP08F1', 'PP08F2', 'PP08J1', 'PP08J2', 'PP08J3', 'P21', 'DECOCUR', 'IDECOCUR', 'RDECOCUR', 'GDECOCUR', 'PDECOCUR', 'ADECOCUR', 'PONDIIO','TOT_P12','P47T', 
                       'DECINDR', 'IDECINDR', 'RDECINDR', 'GDECINDR', 'PDECINDR', 'ADECINDR',
                       'V2_M', 'V3_M', 'V4_M', 'V5_M', 'V8_M', 'V9_M', 'V10_M', 'V11_M', 'V12_M', 'V18_M', 'V19_AM', 'V21_M', 
                       'T_VI','ITF', 'DECIFR', 'IDECIFR', 'RDECIFR', 'GDECIFR', 'PDECIFR', 'ADECIFR','IPCF', 'DECCFR', 'IDECCFR', 'RDECCFR', 'GDECCFR', 'PDECCFR', 'ADECCFR',
                       'ad_equiv', 'ad_equiv_hogar']    
    base_prediccion = df.drop(columns=columns_to_drop)
    if 'ingreso_necesario' in base_prediccion.columns:
        base_prediccion.drop(columns=['ingreso_necesario'])
    base_prediccion = base_prediccion.drop(columns=["CODUSU", "MAS_500", "CH05"])
    base_prediccion = base_prediccion.dropna(thresh = len(base_prediccion), axis = 1)
    base_prediccion["cons"] = 1
    if 'pobre' in base_prediccion.columns:
        return base_prediccion.drop(columns=["pobre"]), base_prediccion.pobre
    else:
        return base_prediccion

Nota:

También podríamos definir estas funciones en un módulo auxiliar.
El módulo auxiliar es un archivo .py (llamémoslo "auxiliar.py") con las funciones. 
Luego, para poder usar las funciones en nuestro notebook lo importaríamos de la siguiente forma:

from auxiliar import preprocesar_eph, generar_base_prediccion

y ver los docstrings de las funciones con help(*nombre_funcion*)

In [7]:
#from auxiliar import preprocesar_eph
#help(preprocesar_eph)

In [ ]:
respondieron, norespondieron = preprocesar_eph(eph_file, ad_equiv_xls)

In [ ]:
respondieron

In [10]:
X, y = generar_base_prediccion(respondieron)

### Evaluando múltiples modelos (y múltiples configuraciones)
Vamos a ver una técnica de programación que nos va a ayudar a escribir código más compacto y fácil de mantener.

#### Particionar datos

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

#### Definir modelos

Vamos a usar diccionarios, con valores un poco más complejos, los algoritmos que queremos usar. Veamos primero un ejemplo sencillo:

In [12]:
def elevar_al_cuadrado(x):
    return x**2

def elevar_al_cubo(x):
    return x**3

def elevar_a_la_cuarta(x):
    return x**4

In [13]:
elevar_a_una_potencia = {
    2: elevar_al_cuadrado,
    3: elevar_al_cubo,
    4: elevar_a_la_cuarta,
}

Usando el diccionario y sus claves, estamos llamando a las funciones que definimos anteriormente

In [ ]:
elevar_a_una_potencia[2]

Ahora veamos cómo funciona esto para iterar sobre modelos

In [15]:
MODELOS = {
    "Análisis de Discriminante Lineal": LinearDiscriminantAnalysis(n_components=1),
    "3 vecinos cercanos": KNeighborsClassifier(n_neighbors=3),
    "Regresión logística": LogisticRegression()
}

In [ ]:
for nombre, modelo in MODELOS.items():
    
    print("Probando modelo: ", nombre)
    
    # Ajustamos el modelo
    modelo.fit(X_train, y_train)
    
    # Realizamos predicción sobre base test
    y_pred = modelo.predict(X_test)
    
    # Calculamos el accuracy y matriz de confusion
    matriz_confusion = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix :')
    print(matriz_confusion)
    
    accuracy = accuracy_score(y_test, y_pred)
    print("La precisión del modelo es: %.3f" % accuracy) 
    print()

Nota: el código de la celda de arriba no es una función (aún), sino un loop. Podrían usar un loop como este para crear la función evalua_metodo del TP4 (definiendo adecuadamente la función con sus parámetros y agregando lo que falta: obtener las métricas pedidas y guardarlas en una colección).

#### Probando distintas configuraciones

Supongamos ahora que queremos probar distintas configuraciones para un mismo modelo, por ejemplo, el número de vecinos $k$ para el modelo de $k$-NN. Para eso, podemos usar un **diccionario** que contenga los valores de los distintos parámetros que definen al modelo (usando como claves los nombres de los params tal como se definen en la función a usar). Por ejemplo,

In [17]:
modelo = KNeighborsClassifier(n_neighbors=5)

es equivalente a  

In [18]:
config = {"n_neighbors": 5}
modelo = KNeighborsClassifier(**config) # **config "expande" el diccionario
#modelo.get_params()   # vemos que el algoritmo está configurado con 5 vecinos

Usando esta técnica, podemos definir estas configuraciones de manera programática, por ejemplo como un diccionario de diccionarios:

In [19]:
knn_configs = {}
for k in range(3, 16):
    clave = "KNN con %s vecinos" % k # Una regla para el nombre del modelo
    config = {"n_neighbors":k}
    knn_configs[clave] = config

In [ ]:
knn_configs

**Alternativa**: Esto mismo se puede hacer definiendo el diccionario _por comprensión_:

(Consiste en llaves rodeando una expresión seguida de la declaración for y luego declaraciones for o if. El resultado será un nuevo diccionario que sale de evaluar la expresión en el contexto de los for o if que le siguen.)

In [21]:
knn_configs = {"KNN con %s vecinos" % k: {"n_neighbors":k} for k in range(3, 16)}

In [ ]:
knn_configs

Ahora, iteremos sobre las distintas configuraciones

In [ ]:
for nombre, config in knn_configs.items():
    print("Probando modelo: ", nombre)
    modelo = KNeighborsClassifier(**config)
    
    # Ajustamos el modelo
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    
    # Calculamos el accuracy y matriz de confusion
    matriz_confusion = confusion_matrix(y_test, y_pred)
    print('Matriz de confusión:')
    print(matriz_confusion)
    
    accuracy = accuracy_score(y_test, y_pred)
    print("La precisión del modelo es: %.3f" % accuracy) 
    print()